In [1]:
%load_ext autoreload
%autoreload 2

# Load EMA weights

In [2]:
from pathlib import Path
from omegaconf import OmegaConf

import torch
from plaid.diffusion import FunctionOrganismDiffusion
from plaid.denoisers import FunctionOrganismDiT, DenoiserKwargs
from plaid.constants import COMPRESSION_INPUT_DIMENSIONS, COMPRESSION_SHORTEN_FACTORS

device = torch.device("cuda")

In [5]:
model_id = "4hdab8dn"

ckpt_dir = Path("/data/lux70/plaid/checkpoints/plaid-compositional") 
model_path = ckpt_dir / model_id / "last.ckpt"
config_path = ckpt_dir / model_id / "config.yaml"

cfg = OmegaConf.load(config_path)

In [6]:
compression_model_id = cfg['compression_model_id']
shorten_factor = COMPRESSION_SHORTEN_FACTORS[compression_model_id]
input_dim = COMPRESSION_INPUT_DIMENSIONS[compression_model_id]

In [7]:
denoiser_kwargs = cfg.denoiser
denoiser_kwargs.pop("_target_")
denoiser = FunctionOrganismDiT(**denoiser_kwargs, input_dim=input_dim)

'plaid.diffusion.FunctionOrganismDiffusion'

In [19]:
# lask.ckpt automatically links to the EMA

ckpt = torch.load(model_path)
# ckpt.keys()

In [51]:
mod_state_dict = {}
for k, v in ckpt['state_dict'].items():
    if k[:16] == "model._orig_mod.":
        mod_state_dict[k[16:]] = v

In [53]:
denoiser.load_state_dict(mod_state_dict)

<All keys matched successfully>

In [54]:
diffusion_kwargs

{'beta_scheduler_name': 'sigmoid', 'beta_scheduler_start': -3, 'beta_scheduler_end': 3, 'beta_scheduler_tau': 1, 'x_downscale_factor': 1.0, 'timesteps': 1000, 'objective': 'pred_v', 'min_snr_loss_weight': True, 'min_snr_gamma': 5, 'x_clip_val': 1.0, 'function_y_cond_drop_prob': 0.1, 'organism_y_cond_drop_prob': 0.1, 'ema_decay': 0.9999, 'lr': 0.0001, 'lr_adam_betas': [0.9, 0.999], 'lr_sched_type': 'cosine_with_restarts', 'lr_num_warmup_steps': 10000, 'lr_num_training_steps': 10000000, 'lr_num_cycles': 1}

In [59]:
diffusion_kwargs['beta_scheduler_name'] = "adm_cosine"
diffusion_kwargs['sampling_timesteps'] = 500

diffusion = FunctionOrganismDiffusion(model=denoiser,**diffusion_kwargs)

In [27]:
from cheap.pretrained import load_model_from_id
cheap_model = load_model_from_id(compression_model_id)
_ = cheap_model.to(device)

from cheap.proteins import LatentToSequence,LatentToStructure
latent_to_sequence = LatentToSequence()
latent_to_sequence.to(device)

latent_to_structure = LatentToStructure()
latent_to_structure.to(device)

Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.


Creating ESMFold...
ESMFold model loaded in 37.96 seconds.


# Sample
Human, carbohydrate metabolic process:

In [60]:
# organism_idx = org_df[org_df.organism_id == "HUMAN"].organism_index.iloc[0]
# function_idx = go_df[go_df.GO_term == "carbohydrate metabolic process"].GO_idx.iloc[0]
# print(organism_idx, function_idx)

organism_idx = 1326
function_idx = 55

In [61]:
device = torch.device("cuda")
diffusion = diffusion.to(device)

In [62]:
N, L = 32, 64
shape = (N, L, input_dim)
organism_y_idxs = torch.full((N,), organism_idx)
function_y_idxs = torch.full((N,), function_idx)

cond_scale = 2

diffusion.sampling_timesteps=500
diffusion
sampled_latent = diffusion.ddim_sample_loop(shape, organism_idx, function_idx, return_all_timesteps=True, cond_scale=cond_scale)

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

In [63]:
print(sampled_latent.shape)
final_sample = sampled_latent[:, -1, :, :]
print(final_sample.shape)

print(final_sample.max(), final_sample.min())

torch.Size([32, 501, 64, 32])
torch.Size([32, 64, 32])
tensor(1.0088, device='cuda:0') tensor(-0.9663, device='cuda:0')


In [64]:
sampled_uncompressed = cheap_model.decode(final_sample, downsampled_mask=None)
print(sampled_uncompressed.min(), sampled_uncompressed.max())

tensor(-0.9442, device='cuda:0') tensor(1.0700, device='cuda:0')


In [65]:
# import pickle as pkl
# with open("test_sample.pkl", "wb") as f:
#     pkl.dump(sampled_uncompressed,f)

In [66]:
from cheap.utils import LatentScaler
latent_scaler = LatentScaler()
sampled_unscaled = latent_scaler.unscale(sampled_uncompressed) 

In [67]:
print(sampled_unscaled.shape)
print(sampled_unscaled.max(), sampled_unscaled.min())

torch.Size([32, 128, 1024])
tensor(2732.1536, device='cuda:0') tensor(-834.0418, device='cuda:0')


In [68]:
sequences = latent_to_sequence.to_sequence(sampled_unscaled)[-1]

In [69]:
sequences[:3]

['ANGGVQHAALAIASGQSVQVVLQIRHIPARDSDYVPALLFSCSPDADCVSCNSIFSNVYQQWEPVGNGIMMLTITLPDTVTLGAGCDVVEFILWDSPPAGEFHCMETPTQECEASPSLPCSIVITVGP',
 'CAMVLLIAGSAAGAANSIHLPSGEKVELHLAVFQNNCDSHDSCRLFNLMPNYLDSQQLTCRDRTELDSTSIESFPKVLSISASQFTPEAASRRDVRWPPDTVEADGDYFLMCGVPDGTCTMGCFVLGV',
 'ARRLLAYAIAVLLVTAFAAAAAAPAQAHTSAPRCQHNFPAASCSRLSYAGHPCNAACRRAGCQNGTLQPFSACDGTGKQLCQEAKPEFRGDPVLMIMDGGTARPCPQAPPGGDYVAPRVQHEAGHAAG']

In [ ]:
pdb_strs, raw_outputs = latent_to_structure.to_structure(sampled_unscaled, return_raw_outputs=True, sequences=sequences, batch_size=32)

(Generating structure):   0%|                                                      | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_outputs['plddt'].mean(dim=-1).mean(dim=-1)

In [ ]:
print(raw_outputs['plddt'].mean())

In [ ]:
import py3Dmol

# for i in range(len(pdb_strs)):
for i in range(0,10): 
    view = py3Dmol.view(width=600, height=600)
    view.addModelsAsFrames(pdb_strs[i])
    
    # Apply the plDDT color scheme
    view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}}})
    # view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    # # Add surface representation with plDDT-based color
    # view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}})

    view.zoomTo()
    view.show()

In [ ]:
from plaid.evaluation import RITAPerplexity

perplexity_calc = RITAPerplexity(device=device)
perplexities = perplexity_calc.batch_eval(sequences)

In [ ]:
perplexities